In [1]:
#!pip install --quiet glvis
import numpy as np
import matplotlib.pyplot as plt
from  bfp import *
import mfem.ser as mfem
#from mfem.common.arg_parser import ArgParser
from glvis import glvis, GlvisData
import seaborn as sns
import pandas as pd
import os

In [9]:
class EDependentCoefficient(mfem.PyCoefficient):
    def __init__(self, xs_t_data, E_start, E_end):
        super(EDependentCoefficient, self).__init__()
        self.E_start = E_start
        self.E_end = E_end
        if isinstance(xs_t_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_t_data)
        else:
            self.constant = False
            self.xs_t_data = xs_t_data
            self.n_groups = len(xs_t_data)
            self.E_bins = np.linspace(E_start, E_end, self.n_groups + 1)

    def EvalValue(self, x):
        if self.constant:
            return self.constant_value
        E = x[1]
        for i in range(self.n_groups - 1):
            if np.isclose(E, self.E_bins[i+1]):
                return float(self.xs_t_data[i+1])
            if E <= self.E_bins[i] and E > self.E_bins[i+1]:
                return float(self.xs_t_data[i])
        return float(self.xs_t_data[-1])

class XDependentCoefficient(mfem.PyCoefficient):
    def __init__(self, xs_t_data, x_start, x_end):
        super(XDependentCoefficient, self).__init__()
        self.x_start = x_start
        self.x_end = x_end
        if isinstance(xs_t_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_t_data)
        else:
            self.constant = False
            self.xs_t_data = xs_t_data
            self.n_groups = len(xs_t_data)
            self.x_bins = np.linspace(x_start, x_end, self.n_groups + 1)

    def EvalValue(self, x):
        if self.constant:
            return self.constant_value

        x_val = x[0]
        for i in range(self.n_groups - 1):
            if np.isclose(x_val, self.x_bins[i+1]):
                return float(self.xs_t_data[i+1])
            if self.x_bins[i] <= x_val < self.x_bins[i+1]:
                return float(self.xs_t_data[i])
        return float(self.xs_t_data[-1])

class MyVectorCoefficient(mfem.VectorConstantCoefficient):
    def __init__(self, mu_coeff, S_coeff):
        super(MyVectorCoefficient, self).__init__(2)
        self.mu_coeff = mu_coeff 
        self.S_coeff = S_coeff

    def EvalValue(self, x):
        mu_val = self.mu_coeff.EvalValue(x) 
        S_val = self.S_coeff.EvalValue(x) 
        return [mu_val, S_val]
    
class InflowCoefficientSN(mfem.PyCoefficient):
    def __init__(self, inflow, mu):
        super(InflowCoefficientSN, self).__init__()
        self.inflow = inflow
        self.mu = mu
    def EvalValue(self, x):
        if self.mu > 0:
            return self.inflow
        else:
            return 0
        
class ConstantVector(mfem.VectorPyCoefficient):
    def __init__(self, constant):
        super(ConstantVector, self).__init__(1)
        self.constant = constant 
    def EvalValue(self, x):
        return self.constant

In [10]:
def test_EDependentCoefficient():
    xs_t_data = [0,1,2,3,4]
    coeff = EDependentCoefficient(xs_t_data, 1, 0.01)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([0.0, 0.604]))
    print(coeff.EvalValue([0.0, 0.406]))
    print(coeff.EvalValue([0.0, 0.208]))
    print(coeff.EvalValue([0.0, 0.01]))

def test_EDependentCoefficient_Constant():
    coeff = EDependentCoefficient(40, 0, 1)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([1.0, 0.604]))
    print(coeff.EvalValue([0.3, 0.406]))
    print(coeff.EvalValue([0.6, 0.208]))
    print(coeff.EvalValue([0.8, 0.01]))

#print("Test for E dependent Coefficient")
#test_EDependentCoefficient()
#print("Test for E dependent Constant Coefficient")
#test_EDependentCoefficient_Constant()

def test_XDependentCoefficientt():
    xs_t_data = [0,1,2,3,4]
    coeff = XDependentCoefficient(xs_t_data, 0, 1)
    print(coeff.EvalValue([0.0, 0.00]))
    print(coeff.EvalValue([0.2, 0.00]))
    print(coeff.EvalValue([0.4, 0.00]))
    print(coeff.EvalValue([0.6, 0.00]))
    print(coeff.EvalValue([0.8, 0.00]))
    print(coeff.EvalValue([1.0, 0.00]))

def test_XDependentCoefficient_Constant():
    coeff = XDependentCoefficient(80, 0.0, 1.0)
    print(coeff.EvalValue([0.0, 0.00]))
    print(coeff.EvalValue([0.2, 0.00]))
    print(coeff.EvalValue([0.4, 0.00]))
    print(coeff.EvalValue([0.6, 0.00]))
    print(coeff.EvalValue([0.8, 0.00]))
    print(coeff.EvalValue([1.0, 0.00]))

#print("Test for X dependent Coefficient")
#test_XDependentCoefficientt()
#print("Test for X dependent Constant Coefficient")
#test_XDependentCoefficient_Constant()

def test_MyVectorCoefficient():
    mu_data = [5]
    mu = XDependentCoefficient(mu_data, 0.0, 1.0)
    S_data = [0,1,2,3,4]
    S = EDependentCoefficient(S_data, 1.0, 0.01)
    coeff = MyVectorCoefficient(mu, S)
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([0.2, 0.604]))
    print(coeff.EvalValue([0.4, 0.406]))
    print(coeff.EvalValue([0.6, 0.208]))
    print(coeff.EvalValue([0.8, 0.01]))

#print("Test for Vector Coefficient")
#test_MyVectorCoefficient()


In [11]:
nx = 10
nE = 10
x_start = 0.0
x_end = 1.0
E_start = 0.0
E_end = 1.0
N_ang = 2
order = 1
inflow = 5.0

#mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
mesh = mfem.Mesh(nx, nE, "QUADRILATERAL", True, x_end, E_end)
#for x in range(ref_levels):
mesh.UniformRefinement()
#mesh.UniformRefinement()
#mesh.UniformRefinement()
#mesh.UniformRefinement()
dim = mesh.Dimension()

# btype = 0-GaussLegendre, 1 - GaussLobatto
fec = mfem.DG_FECollection(order, dim, btype=0)
fes = mfem.FiniteElementSpace(mesh, fec)
Size = fes.GetVSize()
print("Number of unknowns:", Size)
print('Number of finite element unknowns: ' + str(fes.GetVSize()))

mu_vals, w_vals = gauss_legendre_dirs(N_ang)

#ex17'den


ess_tdof_list1 = mfem.intArray()
dir_bdr1 = mfem.intArray(mesh.bdr_attributes.Max())
dir_bdr1.Assign(0)
dir_bdr1[3] = 1
dir_bdr1[1] = 0

ess_tdof_list2 = mfem.intArray()
dir_bdr2 = mfem.intArray(mesh.bdr_attributes.Max())
dir_bdr2.Assign(0)
dir_bdr2[0] = 0
dir_bdr2[2] = 0

S_const = 0.0
xs_t_const = 5.0
#xs_s_const = 1.0
S_arr = np.ones(nE) * S_const
E_arr = np.linspace(E_start, E_end, nE+1)

S_coeff = EDependentCoefficient(S_const, E_start, E_end)
xs_t_coeff = EDependentCoefficient(xs_t_const, E_start, E_end)
#xs_s_coeff = EDependentCoefficient(xs_s_const, E_start, E_end)
#q_coeff = EDependentCoefficient(10.0, E_start, E_end)

a = 5
b = 10
'''A
class QCoefficient(mfem.PyCoefficient):
    def __init__(self, mu):
        mfem.PyCoefficient.__init__(self)
        self.mu = mu
    def EvalValue(self, x):
        return (self.mu*10+5*(5+10*x[0]))
'''

class QCoefficient(mfem.PyCoefficient):
    def __init__(self):
        mfem.PyCoefficient.__init__(self)
    def EvalValue(self, x):
        return (0.577*10+5*(5+10*x[0]))
q_coeff = QCoefficient()

# Solution

alpha = 1.0
beta = 0.5

psi_mu = []
psi_mu_list = []

for mu, w in zip(mu_vals, w_vals):
    print("  Solving for mu =", mu)
    mu_coeff = ConstantCoefficient(mu)
    inflow_coeff = InflowCoefficientSN(inflow, mu)
    #marker = get_marker_for_mu(mesh, mu)
    #v_coeff = MyVectorCoefficient(mu_coeff, S_coeff)
    v_coeff1 = mfem.VectorConstantCoefficient([mu, 0.0])
    v_coeff2 = mfem.VectorConstantCoefficient([0.0, S_const])
    print(f"inflow: {inflow_coeff.EvalValue([inflow, mu])}")
    #print(f"v_coeff: {v_coeff.EvalValue([mu, 0.8])}")
    print(f"sigma_t: {xs_t_coeff.EvalValue([0.0, 0.604])}")


    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff1, 1.0))
    #a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff2, 1.0))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff1, -alpha, beta)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff1, -alpha, beta)), dir_bdr1)
    #a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff2, -alpha, beta)))
    #a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff2, -alpha, beta)), dir_bdr2)
    a.Assemble()
    a.Finalize()
    A = a.SpMat()

    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coeff))
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff1, -alpha, -beta), dir_bdr1)
    #b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff2, -alpha, -beta), dir_bdr2)
    b.Assemble()
    
    psi = mfem.GridFunction(fes)
    psi.Assign(1.0)

    prec = mfem.GSSmoother(A)
    solver = mfem.GMRESSolver()
    solver.SetOperator(A)
    solver.SetPreconditioner(prec)
    solver.SetRelTol(1e-12)
    solver.SetAbsTol(1e-12)
    solver.SetMaxIter(1000)
    solver.SetKDim(30)
    solver.SetPrintLevel(1)
    solver.Mult(b, psi)
    

    g = glvis((mesh, psi), 500,500, keys='ARjmcbp')
    g.set_size(1000, 1000)
    g.render()
    
    #X_arr = mfem.GridFunction(fes)
    #X_arr.Assign(X)
    psi.Save("psi_hs_mu_{:.3f}.gf".format(mu))
    psi_mu.append(psi.GetDataArray())
    psi_mu_list.append((mu, w, psi))


phi = mfem.GridFunction(fes)
phi.Assign(0.0)
for mu, w, X in psi_mu_list:
    phi.Add(w, X)

phi.Save("phi_dg.gf")

    

Number of unknowns: 1600
Number of finite element unknowns: 1600
  Solving for mu = -0.5773502691896257
inflow: 0
sigma_t: 5.0
   Pass :  1   Iteration :   0  ||B r|| = 625.741
   Pass :  1   Iteration :   1  ||B r|| = 253.915
   Pass :  1   Iteration :   2  ||B r|| = 190.96
   Pass :  1   Iteration :   3  ||B r|| = 135.871
   Pass :  1   Iteration :   4  ||B r|| = 106.991
   Pass :  1   Iteration :   5  ||B r|| = 76.0312
   Pass :  1   Iteration :   6  ||B r|| = 58.164
   Pass :  1   Iteration :   7  ||B r|| = 42.9039
   Pass :  1   Iteration :   8  ||B r|| = 32.1763
   Pass :  1   Iteration :   9  ||B r|| = 27.0692
   Pass :  1   Iteration :  10  ||B r|| = 24.1604
   Pass :  1   Iteration :  11  ||B r|| = 22.3164
   Pass :  1   Iteration :  12  ||B r|| = 19.999
   Pass :  1   Iteration :  13  ||B r|| = 18.4346
   Pass :  1   Iteration :  14  ||B r|| = 17.7861
   Pass :  1   Iteration :  15  ||B r|| = 17.7801
   Pass :  1   Iteration :  16  ||B r|| = 17.52
   Pass :  1   Iteration :  

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

  Solving for mu = 0.5773502691896257
inflow: 5.0
sigma_t: 5.0
   Pass :  1   Iteration :   0  ||B r|| = 609.89
   Pass :  1   Iteration :   1  ||B r|| = 254.132
   Pass :  1   Iteration :   2  ||B r|| = 197.961
   Pass :  1   Iteration :   3  ||B r|| = 148.544
   Pass :  1   Iteration :   4  ||B r|| = 120.572
   Pass :  1   Iteration :   5  ||B r|| = 87.572
   Pass :  1   Iteration :   6  ||B r|| = 61.7624
   Pass :  1   Iteration :   7  ||B r|| = 46.7156
   Pass :  1   Iteration :   8  ||B r|| = 36.8611
   Pass :  1   Iteration :   9  ||B r|| = 25.942
   Pass :  1   Iteration :  10  ||B r|| = 20.2683
   Pass :  1   Iteration :  11  ||B r|| = 15.9779
   Pass :  1   Iteration :  12  ||B r|| = 13.6949
   Pass :  1   Iteration :  13  ||B r|| = 9.38797
   Pass :  1   Iteration :  14  ||B r|| = 6.27043
   Pass :  1   Iteration :  15  ||B r|| = 3.18235
   Pass :  1   Iteration :  16  ||B r|| = 2.43529
   Pass :  1   Iteration :  17  ||B r|| = 1.02636
   Pass :  1   Iteration :  18  ||B r|| 

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [12]:
g = glvis((mesh, phi), 500,500, keys='ARjmcbp')
g.set_size(900, 900)
g.render()

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [13]:
g = glvis((mesh, psi), 500,500, keys='rljgac//0')
g.set_size(900, 900)
g.render()


_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [14]:
E = np.linspace(0,1,11)
for i in range(E.shape[0]):
    print(((5+10*i)))

5
15
25
35
45
55
65
75
85
95
105


In [27]:
vertex_array = mesh.GetVertexArray()

n_vertices = mesh.GetNV()
dim = mesh.SpaceDimension()

nodes = np.array(vertex_array, copy=False).reshape(n_vertices, dim)

print("Vertex Coordinates):")
print(nodes)
print(nodes.shape)


Vertex Coordinates):
[[0.   0.  ]
 [0.1  0.  ]
 [0.2  0.  ]
 [0.3  0.  ]
 [0.4  0.  ]
 [0.5  0.  ]
 [0.6  0.  ]
 [0.7  0.  ]
 [0.8  0.  ]
 [0.9  0.  ]
 [1.   0.  ]
 [0.   0.1 ]
 [0.1  0.1 ]
 [0.2  0.1 ]
 [0.3  0.1 ]
 [0.4  0.1 ]
 [0.5  0.1 ]
 [0.6  0.1 ]
 [0.7  0.1 ]
 [0.8  0.1 ]
 [0.9  0.1 ]
 [1.   0.1 ]
 [0.   0.2 ]
 [0.1  0.2 ]
 [0.2  0.2 ]
 [0.3  0.2 ]
 [0.4  0.2 ]
 [0.5  0.2 ]
 [0.6  0.2 ]
 [0.7  0.2 ]
 [0.8  0.2 ]
 [0.9  0.2 ]
 [1.   0.2 ]
 [0.   0.3 ]
 [0.1  0.3 ]
 [0.2  0.3 ]
 [0.3  0.3 ]
 [0.4  0.3 ]
 [0.5  0.3 ]
 [0.6  0.3 ]
 [0.7  0.3 ]
 [0.8  0.3 ]
 [0.9  0.3 ]
 [1.   0.3 ]
 [0.   0.4 ]
 [0.1  0.4 ]
 [0.2  0.4 ]
 [0.3  0.4 ]
 [0.4  0.4 ]
 [0.5  0.4 ]
 [0.6  0.4 ]
 [0.7  0.4 ]
 [0.8  0.4 ]
 [0.9  0.4 ]
 [1.   0.4 ]
 [0.   0.5 ]
 [0.1  0.5 ]
 [0.2  0.5 ]
 [0.3  0.5 ]
 [0.4  0.5 ]
 [0.5  0.5 ]
 [0.6  0.5 ]
 [0.7  0.5 ]
 [0.8  0.5 ]
 [0.9  0.5 ]
 [1.   0.5 ]
 [0.   0.6 ]
 [0.1  0.6 ]
 [0.2  0.6 ]
 [0.3  0.6 ]
 [0.4  0.6 ]
 [0.5  0.6 ]
 [0.6  0.6 ]
 [0.7  0.6 ]
 [0.8  0.6 ]
 [0.

In [43]:
import ctypes

elements = []  # Eleman bağlantılarını depolamak için

for i in range(mesh.GetNE()):
    elem = mesh.GetElement(i)          # i'inci elemanı alıyoruz
    nvertices = elem.GetNVertices()      # Elemandaki vertex sayısı (örneğin, üçgen için 3, dörtgen için 4)
    vertices_ptr = elem.GetVertices()    # Swig pointer (int *) döndürür

    # Pointer adresini tamsayıya çeviriyoruz:
    ptr_address = int(vertices_ptr)
    
    # nvertices elemanlı bir ctypes int dizisi oluşturuyoruz:
    IntArrayType = ctypes.c_int * nvertices
    int_array = IntArrayType.from_address(ptr_address)
    
    # ctypes dizisini Python listesine çeviriyoruz:
    conn = list(int_array)
    elements.append(conn)
i = 0
print("Eleman bağlantıları:")
for conn in elements:
    print(conn)
    print(i)
    i += 1




Eleman bağlantıları:
[0, 121, 341, 124]
0
[121, 1, 122, 341]
1
[341, 122, 12, 123]
2
[124, 341, 123, 11]
3
[11, 123, 342, 127]
4
[123, 12, 125, 342]
5
[342, 125, 23, 126]
6
[127, 342, 126, 22]
7
[22, 126, 343, 130]
8
[126, 23, 128, 343]
9
[343, 128, 34, 129]
10
[130, 343, 129, 33]
11
[23, 131, 344, 128]
12
[131, 24, 132, 344]
13
[344, 132, 35, 133]
14
[128, 344, 133, 34]
15
[12, 134, 345, 125]
16
[134, 13, 135, 345]
17
[345, 135, 24, 131]
18
[125, 345, 131, 23]
19
[1, 136, 346, 122]
20
[136, 2, 137, 346]
21
[346, 137, 13, 134]
22
[122, 346, 134, 12]
23
[2, 138, 347, 137]
24
[138, 3, 139, 347]
25
[347, 139, 14, 140]
26
[137, 347, 140, 13]
27
[13, 140, 348, 135]
28
[140, 14, 141, 348]
29
[348, 141, 25, 142]
30
[135, 348, 142, 24]
31
[14, 143, 349, 141]
32
[143, 15, 144, 349]
33
[349, 144, 26, 145]
34
[141, 349, 145, 25]
35
[3, 146, 350, 139]
36
[146, 4, 147, 350]
37
[350, 147, 15, 143]
38
[139, 350, 143, 14]
39
[4, 148, 351, 147]
40
[148, 5, 149, 351]
41
[351, 149, 16, 150]
42
[147, 351,